In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import altair as alt
alt.renderers.enable('default')
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import_data=pd.read_csv("../input/india-trade-data/2018-2010_import.csv")
export_data=pd.read_csv("../input/india-trade-data/2018-2010_export.csv")

In [3]:
import_data.head()

,HSCode,Commodity,value,country,year
0,5,"PRODUCTS OF ANIMAL ORIGIN, NOT ELSEWHERE SPECI...",0.00,AFGHANISTAN TIS,2018
1,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,12.38,AFGHANISTAN TIS,2018
2,8,EDIBLE FRUIT AND NUTS; PEEL OR CITRUS FRUIT OR...,268.60,AFGHANISTAN TIS,2018
3,9,"COFFEE, TEA, MATE AND SPICES.",35.48,AFGHANISTAN TIS,2018
4,11,PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCH...,NaN,AFGHANISTAN TIS,2018


In [4]:
export_data.head()

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018
3,6,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,0.00,AFGHANISTAN TIS,2018
4,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,1.89,AFGHANISTAN TIS,2018


* HSCode:- The Harmonized System is a standardized numerical method of classifying traded products. It is used by customs authorities around the world to identify products when assessing duties and taxes and for gathering statistics. 

* Commodity:- Name of commodity as per HS2

* value:-  Value of Export or Import according to dataframes

* country:- Country of Export or Import acccording to dataframes

* year:-   Year of Import or Export

In [5]:
import_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76124 entries, 0 to 76123
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   HSCode     76124 non-null  int64  
 1   Commodity  76124 non-null  object 
 2   value      64536 non-null  float64
 3   country    76124 non-null  object 
 4   year       76124 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.9+ MB


In [6]:
export_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137023 entries, 0 to 137022
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   HSCode     137023 non-null  int64  
 1   Commodity  137023 non-null  object 
 2   value      122985 non-null  float64
 3   country    137023 non-null  object 
 4   year       137023 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.2+ MB


*  Both Export and Import Data-Frames have NAN values in the value column and count of NAN values is also less so dropping all rows which contain NAN value

In [7]:
import_data=import_data.dropna()
export_data=export_data.dropna()

In [8]:
unique_import_commodities=import_data.Commodity.unique().tolist()
unique_export_commodities=export_data.Commodity.unique().tolist()
print("No of Unique Import Commodities from 2010 to 2018 =",len(unique_import_commodities))
print("No of Unique Export Commodities from 2010 to 2018 =",len(unique_export_commodities))

No of Unique Import Commodities from 2010 to 2018 = 98
No of Unique Export Commodities from 2010 to 2018 = 98


In [9]:
unique_import_countries=import_data.country.unique().tolist()
unique_export_countries=export_data.country.unique().tolist()
print("No of Countries from which Commodities are imported from 2010 to 2018=",len(unique_import_countries))
print("No of Countries to   which Commodities are exported from 2010 to 2018=",len(unique_export_countries))

No of Countries from which Commodities are imported from 2010 to 2018= 241
No of Countries to   which Commodities are exported from 2010 to 2018= 248


In [10]:
i=import_data.groupby('year',as_index=False).agg({"value":"sum"}).rename(columns={'value':'Import'})
e=export_data.groupby('year',as_index=False).agg({"value":"sum"}).rename(columns={'value':'Export'})
df=pd.merge(i,e,on='year')
df['Growth in Import']=df.set_index('year')['Import'].pct_change().values
df['Growth in Export']=df.set_index('year')['Export'].pct_change().values
df['Deficit']=df['Export']-df['Import']
df['Growth in Import'].fillna(0,inplace=True)
df['Growth in Export'].fillna(0,inplace=True)
df

,year,Import,Export,Growth in Import,Growth in Export,Deficit
0,2010,369762.25,249801.18,0.000000,0.000000,-119961.07
1,2011,489311.81,305948.28,0.323315,0.224767,-183363.53
2,2012,490730.07,300384.32,0.002898,-0.018186,-190345.75
3,2013,450192.99,314388.61,-0.082606,0.046621,-135804.38
4,2014,448026.63,310321.02,-0.004812,-0.012938,-137705.61
5,2015,412537.53,262274.30,-0.079212,-0.154829,-150263.23
6,2016,384350.29,275835.27,-0.068326,0.051705,-108515.02
7,2017,465574.02,303507.85,0.211327,0.100323,-162066.17
8,2018,514071.33,330058.64,0.104167,0.087480,-184012.69


In [11]:
fig=go.Figure(data=([go.Bar(x=df.year,y=df.Import,name='Import',marker={"color":"#50D3E3"}),go.Bar(x=df.year,y=df.Export,marker={"color":"#FF3900"},name="Export")]))
fig.update_layout(title="Grouped Bar Plot of Import and Export from 2010 to 2018",xaxis_title="Year",yaxis_title="Value")
fig.show()

In [12]:
fig=go.Figure(data=([go.Scatter(x=df.year,y=df['Growth in Import'],name='Import',marker={"color":"#99CB00"}),go.Scatter(x=df.year,y=df['Growth in Export'],marker={"color":"#F3B524"},name="Export")]))
fig.update_layout(title="Scatter Plot of Growth Rate in Import and Export from 2010 to 2018",xaxis_title="Year",yaxis_title="Rate")
fig.show()

In [13]:
fig=go.Figure(data=([go.Scatter(x=df.year,y=df['Deficit'],name='Deficit',marker={"color":"#FF38FF"},)]))
fig.update_layout(title="Scatter  Plot of Deficit from 2010 to 2018",xaxis_title="Year",yaxis_title="Deficit")
fig.show()

Reasons for more import and less export may be.

1.Policy failure - after independence we were a closed economy and self sustaining mindset was present. Trade with other countries was very less. India was basically a agricultural economy and manufacturing sector was not developed so much , means we had not much to export. Early leadership was more socialistic in nature than capitalistic so rapid development of manufacturing sector was not present along with that the purchasing power of people was very less so it was difficult to sustain an industry so how could it think of exports. There was no competition as well so quality and technological advancement was not present.

2.LPG reforms - India opened its market for foreign companies , it signed trade agreements with other countries . It reduced tariffs on imports also asked other countries to reduce their tariffs as well. India became member of WTO. Still due to poor manufacturing base it not able to reap the benefits of globalisation. But the service sector got boom after it. Exports of service increased after that and we became a IT superpower.

3.Lack of support to MSME- 40 % of exports is done by MSME sector in India but the condition and the facilities provided to them is very poor.

In [14]:
top_commodities=import_data.groupby(['year','Commodity'],as_index=False).agg({'value':'sum'})
top_2010=top_commodities[top_commodities.year==2010].sort_values('value',ascending=False).head(5)
top_2011=top_commodities[top_commodities.year==2011].sort_values('value',ascending=False).head(5)
top_2012=top_commodities[top_commodities.year==2012].sort_values('value',ascending=False).head(5)
top_2013=top_commodities[top_commodities.year==2013].sort_values('value',ascending=False).head(5)
top_2014=top_commodities[top_commodities.year==2014].sort_values('value',ascending=False).head(5)
top_2015=top_commodities[top_commodities.year==2015].sort_values('value',ascending=False).head(5)
top_2016=top_commodities[top_commodities.year==2016].sort_values('value',ascending=False).head(5)
top_2017=top_commodities[top_commodities.year==2017].sort_values('value',ascending=False).head(5)
top_2018=top_commodities[top_commodities.year==2018].sort_values('value',ascending=False).head(5)
top_commodities=pd.concat([top_2010,top_2011,top_2012,top_2013,top_2014,top_2015,top_2016,top_2017,top_2018])
d={"ORGANIC CHEMICALS":'Organic','MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES.':'Mineral','NATURAL OR CULTURED PEARLS,PRECIOUS OR SEMIPRECIOUS STONES,PRE.METALS,CLAD WITH PRE.METAL AND ARTCLS THEREOF;IMIT.JEWLRY;COIN.':'Natural','NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF.':'Nuclear','ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS, TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS,AND PARTS.':'Electrical'}
top_commodities['Commodity_Name']=top_commodities['Commodity'].map(d)
colors={'Organic':'#FF800C','Mineral':'#4255FF','Nuclear':'#FFF312','Natural':'#A2F312','Electrical':'#FF075B'}
top_commodities['Color']=top_commodities['Commodity_Name'].map(colors)

In [15]:
fig=px.bar(data_frame=top_commodities,x='Commodity_Name',y='value',animation_frame='year',hover_name='Commodity',title="Interactive Bar Plot of Top 5 Import Commodities with year as slider",color='Color')
fig.show()

**Exports structure of 27 - Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes - to India in 2020**

* 61% (64 billion US$): 2709 - Petroleum oils and oils obtained from bituminous minerals, crude.
    
* 15.2% (15.8 billion US$): 2701 - Coal; briquettes, ovoids and similar solid fuels manufactured from coal.
    
* 14.6% (15.2 billion US$): 2711 - Petroleum gases and other gaseous hydrocarbons.
    
* 5.63% (5.87 billion US$): 2710 - Petroleum oils and oils obtained from bituminous minerals, other than crude; preparations not elsewhere specified or included, containing by weight 70 % or more of petroleum oils or of oils obtained from bituminous minerals, these oils being the basic constituents of the preparations; waste oils.
    
* 1.58% (1.64 billion US$): 2713 - Petroleum coke, petroleum bitumen and other residues of petroleum oils or of oils obtained from bituminous minerals.
    
* 0.598% (625 million US$): 2704 - Coke and semi?coke of coal, of lignite or of peat, whether or not agglomerated; retort carbon.
    
* 0.366% (382 million US$): 2707 - Oils and other products of the distillation of high temperature coal tar; similar products in which the weight of the aromatic constituents exceeds that of the non-aromatic constituents.
    
* 0.097% (101 million US$): 2712 - Petroleum jelly; paraffin wax, micro-crystalline petroleum wax, slack wax, ozokerite, lignite wax, peat wax, other mineral waxes, and similar products obtained by synthesis or by other processes, whether or not coloured.
    
* 0.013% (14.1 million US$): 2708 - Pitch and pitch coke, obtained from coal tar or from other mineral tars.
    
* 0.002% (2.6 million US$): 2714 - Bitumen and asphalt, natural; bituminous or oil shale and tar sands; asphaltites and asphaltic rocks.*

**Exports structure of 7116 - Articles of natural or cultured pearls, precious or semi-precious stones (natural, synthetic or reconstructed). - from India in 2020**

* 99% (19.1 million US$): 711620 - Articles of precious/semi-precious stones (nat./synth./reconstructed)

* 0.397% (76 thousand US$): 711610 - Articles of nat./cultured pearls

**Exports structure of 84 - Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof - from India in 2020**

* 15.8% (2.85 billion US$): 8411 - Turbo-jets, turbo-propellers and other gas turbines.

* 7.37% (1.32 billion US$): 8481 - Taps, cocks, valves and similar appliances for pipes, boiler shells, tanks, vats or the like, including pressure-reducing valves and thermostatically controlled valves.

* 6.13% (1.1 billion US$): 8483 - Transmission shafts (including cam shafts and crank shafts) and cranks; bearing housings and plain shaft bearings; gears and gearing; ball or roller screws; gear boxes and other speed changers, including torque converters; flywheels and pulleys, including pulley blocks; clutches and shaft couplings (including universal joints).

* 5.09% (916 million US$): 8409 - Parts suitable for use solely or principally with the engines of heading 84.07 or 84.08.

* 4.57% (821 million US$): 8413 - Pumps for liquids, whether or not fitted with a measuring device; liquid elevators.

* 4.55% (819 million US$): 8408 - Compression-ignition internal combustion piston engines (diesel or semi-diesel engines).

* 4.47% (804 million US$): 8414 - Air or vacuum pumps, air or other gas compressors and fans; ventilating or recycling hoods incorporating a fan, whether or not fitted with filters.

* 4.06% (730 million US$): 8479 - Machines and mechanical appliances having individual functions, not specified or included elsewhere in this Chapter.

* 3.41% (614 million US$): 8421 - Centrifuges, including centrifugal dryers; filtering or purifying machinery and apparatus, for liquids or gases.

* 3.27% (587 million US$): 8431 - Parts suitable for use solely or principally with the machinery of headings 84.25 to 84.30.


**Exports structure of 8543 - Electrical machines and apparatus, having individual functions, not specified or included elsewhere in this Chapter. - from India in 2020**


* 53% (86 million US$): 854389 - Other electrical machines & app., having individual functions, n.e.s. in Ch.85

* 22% (36 million US$): 854390 - Parts of the mach. & app. of 85.43

* 18.6% (30 million US$): 854381 - Proximity cards & tags

* 3.65% (5.92 million US$): 854320 - Signal generators

* 1.56% (2.53 million US$): 854330 - Machines & app. for electroplating/electrolysis/electrophoresis

* 0.246% (400 thousand US$): 854319 - Particle accelerators other than ion implanters for doping semiconductor mats.







In [16]:
top_commodities=export_data.groupby(['year','Commodity'],as_index=False).agg({'value':'sum'})
top_2010=top_commodities[top_commodities.year==2010].sort_values('value',ascending=False).head(5)
top_2011=top_commodities[top_commodities.year==2011].sort_values('value',ascending=False).head(5)
top_2012=top_commodities[top_commodities.year==2012].sort_values('value',ascending=False).head(5)
top_2013=top_commodities[top_commodities.year==2013].sort_values('value',ascending=False).head(5)
top_2014=top_commodities[top_commodities.year==2014].sort_values('value',ascending=False).head(5)
top_2015=top_commodities[top_commodities.year==2015].sort_values('value',ascending=False).head(5)
top_2016=top_commodities[top_commodities.year==2016].sort_values('value',ascending=False).head(5)
top_2017=top_commodities[top_commodities.year==2017].sort_values('value',ascending=False).head(5)
top_2018=top_commodities[top_commodities.year==2018].sort_values('value',ascending=False).head(5)
top_commodities=pd.concat([top_2010,top_2011,top_2012,top_2013,top_2014,top_2015,top_2016,top_2017,top_2018])
d={'NATURAL OR CULTURED PEARLS,PRECIOUS OR SEMIPRECIOUS STONES,PRE.METALS,CLAD WITH PRE.METAL AND ARTCLS THEREOF;IMIT.JEWLRY;COIN.':'Jewelry','MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES.':'Mineral','MISCELLANEOUS GOODS.':'MISCELLANEOUS GOODS.','ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS, TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS,AND PARTS.':'Electrical Machinery','VEHICLES OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK, AND PARTS AND ACCESSORIES THEREOF.':'Vehicle Parts','ORGANIC CHEMICALS':'Organic Chemicals','NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF.':'Nuclear Reactors','PHARMACEUTICAL PRODUCTS':'PHARMACEUTICAL PRODUCTS'}
top_commodities['Commodity_Name']=top_commodities['Commodity'].map(d)
colors={'Jewelry':'#FF800C','Mineral':'#4255FF','MISCELLANEOUS GOODS.':'#FFF312','Electrical Machinery':'#A2F312','Vehicle Parts':'#FF075B','Organic Chemicals':"#6EC9CB",'Nuclear Reactors':'#B95C46','PHARMACEUTICAL PRODUCTS':'#B9F846'}
top_commodities['Color']=top_commodities['Commodity_Name'].map(colors)

In [17]:
fig=px.bar(data_frame=top_commodities,x='Commodity_Name',y='value',animation_frame='year',hover_name='Commodity',title="Interactive Bar Plot of Top 5 Export Commodities with year as slider",color='Color')
fig.show()

**Exports structure of 7116 - Articles of natural or cultured pearls, precious or semi-precious stones (natural, synthetic or reconstructed). - from India in 2020**

* 99% (19.1 million US$): 711620 - Articles of precious/semi-precious stones.

*  0.397% (76 thousand US$): 711610 - Articles of nat./cultured pearls

**Exports structure of 27 - Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes - from India in 2020**

* 94% (26 billion US$): 2710 - Petroleum oils and oils obtained from bituminous minerals, other than crude; preparations not elsewhere specified or included, containing by weight 70 % or more of petroleum oils or of oils obtained from bituminous minerals, these oils being the basic constituents of the preparations; waste oils.

* 1.77% (491 million US$): 2716 - Electrical energy. (optional heading).

* 1.49% (412 million US$): 2707 - Oils and other products of the distillation of high temperature coal tar; similar products in which the weight of the aromatic constituents exceeds that of the non-aromatic constituents.

* 0.868% (239 million US$): 2711 - Petroleum gases and other gaseous hydrocarbons.

* 0.467% (129 million US$): 2713 - Petroleum coke, petroleum bitumen and other residues of petroleum oils or of oils obtained from bituminous minerals.

* 0.257% (71 million US$): 2712 - Petroleum jelly; paraffin wax, micro-crystalline petroleum wax, slack wax, ozokerite, lignite wax, peat wax, other mineral waxes, and similar products obtained by synthesis or by other processes, whether or not coloured.

* 0.249% (68 million US$): 2701 - Coal; briquettes, ovoids and similar solid fuels manufactured from coal.

* 0.087% (24 million US$): 2704 - Coke and semi?coke of coal, of lignite or of peat, whether or not agglomerated; retort carbon.

* 0.04% (11.1 million US$): 2708 - Pitch and pitch coke, obtained from coal tar or from other mineral tars.

* 0.015% (4.18 million US$): 2702 - Lignite, whether or not agglomerated, excluding jet.


**Exports structure of 8520 - Magnetic tape recorders and other sound recording apparatus, whether or not incorporating a sound reproducing device. - from India in 2020**

* 99% (36 thousand US$): 852033 - Magnetic tape recorders incorp. sound repr. app. (excl. of 8520.10 & 8520.20), cassette-type.

* 0.193% (70 US$): 852020 - Telephone answering machines


**Exports structure of 84 - Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof - from India in 2020**

* 15.8% (2.85 billion US$): 8411 - Turbo-jets, turbo-propellers and other gas turbines.

* 7.37% (1.32 billion US$): 8481 - Taps, cocks, valves and similar appliances for pipes, boiler shells, tanks, vats or the like, including pressure-reducing valves and thermostatically controlled valves.

* 6.13% (1.1 billion US$): 8483 - Transmission shafts (including cam shafts and crank shafts) and cranks; bearing housings and plain shaft bearings; gears and gearing; ball or roller screws; gear boxes and other speed changers, including torque converters; flywheels and pulleys, including pulley blocks; clutches and shaft couplings (including universal joints).

* 5.09% (916 million US$): 8409 - Parts suitable for use solely or principally with the engines of heading 84.07 or 84.08.

* 4.57% (821 million US$): 8413 - Pumps for liquids, whether or not fitted with a measuring device; liquid elevators.

* 4.55% (819 million US$): 8408 - Compression-ignition internal combustion piston engines (diesel or semi-diesel engines).

* 4.47% (804 million US$): 8414 - Air or vacuum pumps, air or other gas compressors and fans; ventilating or recycling hoods incorporating a fan, whether or not fitted with filters.

* 4.06% (730 million US$): 8479 - Machines and mechanical appliances having individual functions, not specified or included elsewhere in this Chapter.

* 3.41% (614 million US$): 8421 - Centrifuges, including centrifugal dryers; filtering or purifying machinery and apparatus, for liquids or gases.

* 3.27% (587 million US$): 8431 - Parts suitable for use solely or principally with the machinery of headings 84.25 to 84.30.







In [18]:
df=import_data.groupby(['year','country'],as_index=False).agg({'value':'sum'})
top_countries=import_data.groupby('country',as_index=False).agg({'value':'sum'}).sort_values('value',ascending=False).head(5)['country'].tolist()
print("Top Countries from which Commodities are imported =",top_countries)
china=df[df['country']==top_countries[0]]
uae=df[df['country']==top_countries[1]]
saudi_arab=df[df['country']==top_countries[2]]
usa=df[df['country']==top_countries[3]]
switzerland=df[df['country']==top_countries[4]]

Top Countries from which Commodities are imported = ['CHINA P RP', 'U ARAB EMTS', 'SAUDI ARAB', 'U S A', 'SWITZERLAND']


In [19]:
alt.Chart(china,title='Import from China from 2010-2018').mark_line(color='#FF3D46').encode(x=alt.X('year',axis=alt.Axis(title='YEAR')),y=alt.Y('value',axis=alt.Axis(title='YEAR'))).properties(width=500).configure_axis(labelFontSize=20,titleFontSize=20).configure_title(fontSize=20)


alt.Chart(...)

In [20]:
alt.Chart(usa,title='Import from USA from 2010-2018').mark_line(color='#00FF1D').encode(x=alt.X('year',axis=alt.Axis(title='YEAR')),y=alt.Y('value',axis=alt.Axis(title='YEAR'))).properties(width=500).configure_axis(labelFontSize=20,titleFontSize=20).configure_title(fontSize=20)

alt.Chart(...)

In [21]:
alt.Chart(switzerland,title='Import from Switzerland from 2010-2018').mark_line(color='#00B4FF').encode(x=alt.X('year',axis=alt.Axis(title='YEAR')),y=alt.Y('value',axis=alt.Axis(title='YEAR'))).properties(width=500).configure_axis(labelFontSize=20,titleFontSize=20).configure_title(fontSize=20)

alt.Chart(...)

In [22]:
df=export_data.groupby(['year','country'],as_index=False).agg({'value':'sum'})
top_countries=export_data.groupby('country',as_index=False).agg({'value':'sum'}).sort_values('value',ascending=False).head(5)['country'].tolist()
print("Top Countries from which Commodities are imported =",top_countries)
usa=df[df['country']==top_countries[0]]
uae=df[df['country']==top_countries[1]]
china=df[df['country']==top_countries[2]]
hongkong=df[df['country']==top_countries[3]]
singapore=df[df['country']==top_countries[4]]

Top Countries from which Commodities are imported = ['U S A', 'U ARAB EMTS', 'CHINA P RP', 'HONG KONG', 'SINGAPORE']


In [23]:
alt.Chart(usa,title='Export to USA from 2010-2018').mark_line(color='#842600').encode(x=alt.X('year',axis=alt.Axis(title='YEAR')),y=alt.Y('value',axis=alt.Axis(title='YEAR'))).properties(width=500).configure_axis(labelFontSize=20,titleFontSize=20).configure_title(fontSize=20)

alt.Chart(...)

In [24]:
alt.Chart(uae,title='Export to UAE from 2010-2018').mark_line(color='#FECE00').encode(x=alt.X('year',axis=alt.Axis(title='YEAR')),y=alt.Y('value',axis=alt.Axis(title='YEAR'))).properties(width=500).configure_axis(labelFontSize=20,titleFontSize=20).configure_title(fontSize=20)

alt.Chart(...)

In [25]:
alt.Chart(singapore,title='Export to Singapore from 2010-2018',).mark_line(color='#00A407').encode(x=alt.X('year',axis=alt.Axis(title='YEAR')),y=alt.Y('value',axis=alt.Axis(title='VALUE'))).properties(width=500).configure_axis(labelFontSize=20,titleFontSize=20).configure_title(fontSize=20)

alt.Chart(...)

**UAE**
* India's exports to the UAE are well diversified with a large basket. India's major export items to the UAE are: Petroleum Products, Precious Metals, Stones, Gems & Jewellery, Minerals, Food Items (Cereals, Sugar, Fruits & Vegetables, Tea, Meat, and Seafood), Textiles (Garments, Apparel, Synthetic fibre, Cotton, Yarn) and Engineering & Machinery Products and Chemicals. 

* India's major import items from the UAE are:  Petroleum and Petroleum Products, Precious Metals, Stones, Gems & Jewellery, Minerals, Chemicals, Wood & Wood Products.   India also imported 21.83 MMT (US$ 10,927.52 Million) of crude oil from UAE in 2019-20.

**USA**

* India's Major export items to USA are:-Medical Appliances,Leather Goods,Textile Goods,Rice,Spices,Pulses,Jewelry,Oils,Machinery.

* India's Major import items to USA are:- Cashew Nuts,Chemical Fertilizers,Edible Oils,Food Grains, Iron and Steel,Medicines.


**China**

* India's Major export items to China are:- Cotton yarn,Iron core, Organic Chemicals,Mineral Fuels,Fish,Salts.

* India's Major import items from china are:- Electronics,Heavy-Machinery,Medicines,Consumer products

In [26]:
import_countries=import_data.groupby('country',as_index=False).agg({'value':'sum'})
px.choropleth(data_frame=import_countries,locations='country',color='value',locationmode='country names')

In [27]:
export_countries=export_data.groupby('country',as_index=False).agg({'value':'sum'})
px.choropleth(data_frame=export_countries,locations='country',color='value',locationmode='country names')